Simple word2vec model with 200 features for each utterance, not considering other features (speaker etc.)

Doc2Vec model for all chats

In [188]:
import os
import json
import gensim

# path = r"C:\Users\user\git\NLPProj\Frames-dataset\chats"


def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def gen_chat_model(label_dict):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats = []
    avg_chat_len = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
                sents.extend( turn['text'].split() )
                avg_chat_len += 1
        tagged_chat = gensim.models.doc2vec.TaggedDocument(sents, label_dict[filename])
        chats.append( tagged_chat )
    avg_chat_len /= len(chats)
    print("average number of sents in chat: " + str(avg_chat_len))
    return gensim.models.Doc2Vec(chats, size=100, window=8, min_count=1, workers=4)

# d = build_label_dict()
# chat_model = gen_chat_model(d)

Prepare input vectors from first parts of chats and labels from dictionary

In [190]:
def infer_vectors(chat_model, label_dict, num_of_sents_from_start):
    '''infer vectors for every sentence in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.extend( chat['turns'][i]['text'].split() )
#             for turn in chat['turns']:
#                 sents.extend( turn['text'].split() )
        sent_vec = chat_model.infer_vector( sents )
        chats_vecs.append( sent_vec )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

One-hot model for words

In [ ]:
def gen_one_hot_dict():
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    index_dict = {}
    i = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        if 'turns' in chat:
            for turn in chat['turns']:
                for word in turn['text'].split():
                    if word.strip() not in index_dict:
                        index_dict[word.strip()] = i
                        i += 1
    return index_dict

def infer_vectors_from_one_hot(index_dict, label_dict, num_of_sents_from_start):
    '''infer vectors for every sentence in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.extend( [index_dict[word.strip()] for word in chat['turns'][i]['text'].split()] )
                
        sent_vec = 
        chats_vecs.append( sent_vec )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

Simple LSTM network - does not perform well

In [199]:
import numpy as np
from keras.layers import Input, Dense, merge, Embedding, RNN
from keras.models import Model
import keras
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = x_train.shape
train_size = 1000

# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = 6
X, Y = infer_vectors(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]


# Define the model
model = Sequential()
# model.add(Dense(80, input_dim=100, activation='relu'))
input_layer = Dense(80, input_dim=100, activation='relu')

# one attention layer
# attention_probs = Dense(100, activation='softmax', name='attention_probs')(input_layer)
# attention_mul = merge([input_layer, attention_probs], output_shape=(80,), name='attention_mul', mode='mul')
# model.add(attention_mul)
# model.add(input_layer)
# model.add(Dense(60, activation='relu'))
model.add(Embedding(100, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) # recurrent
model.add(Dense(64, activation='softmax', name='attention_probs'))
model.add(Dense(1, activation='sigmoid'))

# Compile and fit
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Test the model
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

average number of sents in chat: 14.598977355734112
Epoch 1/100
1000/1000 [==============================] - 10s - loss: 0.6827 - acc: 0.5760    
Epoch 2/100
1000/1000 [==============================] - 9s - loss: 0.6701 - acc: 0.6140     
Epoch 3/100
1000/1000 [==============================] - 9s - loss: 0.6683 - acc: 0.6140     
Epoch 4/100
1000/1000 [==============================] - 10s - loss: 0.6679 - acc: 0.6140    
Epoch 5/100
1000/1000 [==============================] - 9s - loss: 0.6676 - acc: 0.6140     
Epoch 6/100
1000/1000 [==============================] - 9s - loss: 0.6673 - acc: 0.6140     
Epoch 7/100
1000/1000 [==============================] - 9s - loss: 0.6673 - acc: 0.6140     
Epoch 8/100
1000/1000 [==============================] - 10s - loss: 0.6671 - acc: 0.6140    
Epoch 9/100
1000/1000 [==============================] - 10s - loss: 0.6670 - acc: 0.6140    
Epoch 10/100
1000/1000 [==============================] - 10s - loss: 0.6670 - acc: 0.6140    
Epoch 1

RNN network with attention - does not perform well currently

In [210]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, merge, Embedding, recurrent
from keras.models import Model
import keras
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = x_train.shape
train_size = 1000

# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = 6
X, Y = infer_vectors(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]



# model = Sequential()
# # model.add(Dense(80, input_dim=100, activation='relu'))
# input_layer = Dense(80, input_dim=100, activation='relu')


RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 100
SENT_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40

# Define the model
sentence = Input(shape=(input_dims[1],), dtype='int32')
encoded_sentence = Embedding(100, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = layers.Dropout(0.3)(encoded_sentence)
attention_probs = Dense(100, activation='softmax', name='attention_probs')(encoded_sentence)# one attention layer


merged = layers.add([encoded_sentence, attention_probs])
merged = RNN(EMBED_HIDDEN_SIZE)(merged)
merged = layers.Dropout(0.3)(merged)
preds = Dense(1, activation='sigmoid')(merged)

model = Model([sentence], preds)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print('Training')
model.fit([x_train], y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([x_test], y_test,
                           batch_size=BATCH_SIZE)

print('\nTest loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

average number of sents in chat: 14.598977355734112
Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 6s - loss: 0.6687 - acc: 0.6137 - val_loss: 0.7034 - val_acc: 0.5400 loss: 0.6745 - ac
Epoch 2/40
950/950 [==============================] - 6s - loss: 0.6673 - acc: 0.6179 - val_loss: 0.6933 - val_acc: 0.5400
Epoch 3/40
950/950 [==============================] - 7s - loss: 0.6710 - acc: 0.6179 - val_loss: 0.6932 - val_acc: 0.5400
Epoch 4/40
950/950 [==============================] - 7s - loss: 0.6690 - acc: 0.6179 - val_loss: 0.7068 - val_acc: 0.5400
Epoch 5/40
950/950 [==============================] - 7s - loss: 0.6668 - acc: 0.6179 - val_loss: 0.7021 - val_acc: 0.5400
Epoch 6/40
950/950 [==============================] - 7s - loss: 0.6653 - acc: 0.6179 - val_loss: 0.6973 - val_acc: 0.5400
Epoch 7/40
950/950 [==============================] - 7s - loss: 0.6675 - acc: 0.6179 - val_loss: 0.6959 - val_acc: 0.5400
Epoch 8/40
950/